In [7]:
# CODE FOR QUESTION 1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
ticker_symbol='TCS.NS'
ticket=yf.Ticker(ticker_symbol)
data=ticket.history(period="1y", start="2023-01-01", end="2023-12-31", interval="1d")
import warnings
warnings.filterwarnings("ignore")
# Simple Moving Average (SMA)
def mean(data, i, period):
  if i < period - 1:
    return None
  else:
    return data['Close'].iloc[i - period + 1:i + 1].mean()

data['SMA_30']=None
for i in range(0,len(data)):
  data['SMA_30'].iloc[i]=mean(data, i, 30)


# EXPONENTIAL MOVING AVERAGE
def ema(day):
  s = 2/(1+day)
  for i in range (0,len(data)):
    if i==0:
      data['EMA'].iloc[i]=data['Close'].iloc[i]
    else:
      data['EMA'].iloc[i]=s*(data['Close'].iloc[i])+(1-s)*data['EMA'].iloc[i-1]
  #return data
data['EMA']=None
ema(12)

#Part 1
#Bollinger band
def mean(data, i, period):
  if i < period - 1:
    return None
  else:
    return data['Close'].iloc[i - period + 1:i + 1].mean()

data['SMA_30']=None
for i in range(0,len(data)):
  data['SMA_30'].iloc[i]=mean(data, i, 30)
def moving_std(data, i, period):
  if i < period - 1:
    return None
  else:
    return data['Close'].iloc[i - period + 1:i + 1].std()

data['MSD_30']=None
for i in range(0,len(data)):
  data['MSD_30'].iloc[i]=moving_std(data, i, 30)
def bollinger_band(data,period):
  data['Middle_Band']=data['SMA_30']
  data['Upper_Band']=data['SMA_30']+(2*data['MSD_30'])
  data['Lower_Band']=data['SMA_30']-(2*data['MSD_30'])
  return data


bollinger_band(data,30)
import matplotlib.pyplot as plt
def ema(day):
  s = 2/(1+day)
  data['EMA']=None
  for i in range (0,len(data)):
    if i==0:
      data['EMA'].iloc[i]=data['Close'].iloc[i]
    else:
      data['EMA'].iloc[i]=s*(data['Close'].iloc[i])+(1-s)*data['EMA'].iloc[i-1]
  return data['EMA']
def MACD(data,period1,period2,period3):
  data['EMA_12']=ema(period1)
  data['EMA_26']=ema(period2)
  data['MACD']=None
  for i in range(len(data)):
        if data['EMA_12'][i] is not None and data['EMA_26'][i] is not None:
           data['MACD'].iloc[i] = data['EMA_12'].iloc[i] - data['EMA_26'].iloc[i]
        else:
            data['MACD'].iloc[i] = None
  data['Signal']=None
  x=2/(1+period3)
  for i in range (0,len(data)):
    if i==0:
      data['Signal'].iloc[i]=data['MACD'].iloc[i]
    else:
      data['Signal'].iloc[i]=x*data['MACD'].iloc[i]+(1-x)*data['Signal'].iloc[i-1]
MACD(data,12,26,9)

#Stochastic Oscillator(SO)
def SO(data,period):
  data['Lowest_low']=data['Low'].rolling(window=period).min()
  data['Maximum_high']=data['High'].rolling(window=period).max()
  data['stochastic oscillator']=(data['Close']-data['Lowest_low'])/(data['Maximum_high']-data['Lowest_low'])*100
  return data

SO(data,14)

#Relative Strength Index
def rsi(data,length):
  data['delta']=data['Close'].diff()
  data['gain']=data['delta'].where(data['delta']>0,0)
  data['loss']=data['delta'].where(data['delta']<0,0)
  data['avg_gain']=data['gain'].rolling(window=length).mean()
  data['avg_loss']=data['loss'].rolling(window=length).mean()
  data['rs']=data['avg_gain']/data['avg_loss']
  data['rsi']=100-(100/(1+data['rs']))
  return data
rsi(data,14)

#Average True Range
def atr(data,period):
  data['H-L']=data['High']-data['Low']
  data['H-Cp']=data['High']-data['Close'].shift(1)
  data['L-Cp']=data['Low']-data['Close'].shift(1)
  data['TR']=data[['H-L','H-Cp','L-Cp']].max(axis=1)
  data['ATR']=data['TR'].rolling(window=period).mean()
  return data
atr(data,14)

#Average Directional Index
#Average Directional Index
def dir_m(data):
    data['+DM'] = data['High']-data['High'].shift(1)
    data['+DM'] = data['+DM'].apply(lambda x: x if x > 0 else 0)
    data['-DM'] = data['Low'].shift(1) - data['Low']
    data['+DM'] = data['+DM'].apply(lambda x: x if x > 0 else 0)
def emp(data, period):
    s = 2 / (1 + period)
    data['EMP'] = 0
    for i in range(1, len(data)):
        data['EMP'].iloc[i] = s * (data['+DM'].iloc[i]) + (1 - s) * data['EMP'].iloc[i - 1]

def emn(data, period):
    s = 2 / (1 + period)
    data['EMN'] = 0
    for i in range(1, len(data)):
        data['EMN'].iloc[i] = s * (data['-DM'].iloc[i]) + (1 - s) * data['EMN'].iloc[i - 1]

def dir_m_index(data):
    data['+DI'] = 100 * data['EMP'] / data['ATR']
    data['-DI'] = 100 * data['EMN'] / data['ATR']
    data['+DI'] = data['+DI'].fillna(0)
    data['-DI'] = data['-DI'].fillna(0)
    data['DX'] = (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI'])) * 100
    data['ADX'] = data['DX'].rolling(window=14).mean()
    data['ADX'] = data['ADX'].fillna(0)

dir_m(data)
emp(data, 14)
emn(data, 14)
dir_m_index(data)





In [8]:
# CODE FOR QUESTION 2
#Signal for bollimger band
def signal_bollinger_bands(data):
  data['Signal_Bollinger']=0
  data.loc[data['Close'] > data['Upper_Band'], 'Signal_Bollinger'] = -1  # Sell signal
  data.loc[data['Close'] < data['Lower_Band'], 'Signal_Bollinger'] = 1 # Buy signal
  buy_signals = len(data[data['Signal_Bollinger'] == 1])
  sell_signals = len(data[data['Signal_Bollinger'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")  # Buy signal
signal_bollinger_bands(data)
def signal_MACD(data):
  data['Signal_MACD']=0
  data.loc[data['MACD'] > data['Signal'], 'Signal_MACD'] = 1
  data.loc[data['MACD'] < data['Signal'], 'Signal_MACD'] = -1
  buy_signals = len(data[data['Signal_MACD'] == 1])
  sell_signals = len(data[data['Signal_MACD'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data
signal_MACD(data)
def signal_Stochastic_Oscillator(data):
  data['Signal_SO']=0
  data.loc[data['stochastic oscillator'] > 80, 'Signal_SO'] = -1
  data.loc[data['stochastic oscillator'] < 20, 'Signal_SO'] = 1
  buy_signals = len(data[data['Signal_SO'] == 1])
  sell_signals = len(data[data['Signal_SO'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data
signal_Stochastic_Oscillator(data)

def signal_atr(data):
  multiplier_atr=1.4
  data['Buy'] = data['Close'].shift(1) + (multiplier_atr* data['ATR'])
  data['Sell'] = data['Close'].shift(1) - (multiplier_atr* data['ATR'])
  data['Signal_ATR']=0
  data.loc[data['Close'] > data['Buy'], 'Signal_ATR'] = 1
  data.loc[data['Close'] < data['Sell'], 'Signal_ATR'] = -1
  buy_signals = len(data[data['Signal_ATR'] == 1])
  sell_signals = len(data[data['Signal_ATR'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data['Signal_ATR']
signal_atr(data)
def avg_dir_index(data):
  minimum_adx=25
  data['Signal_DI']=0
  data.loc[(data['+DI'] > data['-DI']) & (data['ADX']>minimum_adx) , 'Signal_DI'] = 1
  data.loc[(data['+DI'] < data['-DI']) & (data['ADX']>minimum_adx) , 'Signal_DI'] = -1
  buy_signals = len(data[data['Signal_DI'] == 1])
  sell_signals = len(data[data['Signal_DI'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data['Signal_DI']
avg_dir_index(data)
#RSI
def rsi(data):
  data['Signal_rsi']=0
  data.loc[data['rsi'] < 30 , 'Signal_rsi'] = 1
  data.loc[data['rsi'] > 70 , 'Signal_rsi'] = -1
  buy_signals = len(data[data['Signal_rsi'] == 1])
  sell_signals = len(data[data['Signal_rsi'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data['Signal_rsi']
rsi(data)




Number of Buy signals: 10
Number of Sell signals: 13
Number of Buy signals: 140
Number of Sell signals: 104
Number of Buy signals: 48
Number of Sell signals: 71
Number of Buy signals: 9
Number of Sell signals: 4
Number of Buy signals: 179
Number of Sell signals: 39
Number of Buy signals: 89
Number of Sell signals: 143


,Signal_rsi
Date,
2023-01-02 00:00:00+05:30,0
2023-01-03 00:00:00+05:30,0
2023-01-04 00:00:00+05:30,0
2023-01-05 00:00:00+05:30,0
2023-01-06 00:00:00+05:30,0
...,...
2023-12-22 00:00:00+05:30,-1
2023-12-26 00:00:00+05:30,-1
2023-12-27 00:00:00+05:30,-1


In [9]:
initial_capital=1000
stop_profit=5
stop_loss=3
data['trade_Signal'] = (
    0.15 * data['Signal_Bollinger'] +
    0.25 * data['Signal_MACD'] +
    0.15 * data['Signal_SO'] +
    0.15 * data['Signal_rsi'] +
    0.20 * data['Signal_ATR'] +
    0.10 * data['Signal_DI']

)
data['Combined_Signal'] = 0
data.loc[data['trade_Signal'] > 0, 'Combined_Signal'] = 1
data.loc[data['trade_Signal'] < 0, 'Combined_Signal'] = -1

In [28]:
class Backtest:
    def __init__(self, data, stop_loss_percent, take_profit_percent, initial_capital, risk_free_rate):
        self.data = data
        self.stop_loss_percent = stop_loss_percent
        self.take_profit_percent = take_profit_percent
        self.initial_capital = initial_capital
        self.trade_log = []
        self.results = {
            "total_trades": 0,
            "winning_trades": 0,
            "losing_trades": 0,
            "stop_loss_hits": 0,
            "take_profit_hits": 0,
        }
        self.equity_curve = []
        self.returns = []
        self.risk_free_rate = risk_free_rate

    def execute_backtest(self):
        capital = self.initial_capital
        position = 0
        entry_price = 0

        for i in range(1, len(self.data)):
            signal = self.data['Combined_Signal'].iloc[i]
            current_price = self.data['Close'].iloc[i]

            if signal == 1 and position == 0:
                position = capital / current_price
                entry_price = current_price
                self.trade_log.append({"action": "buy", "price": entry_price})
            elif position > 0:
                stop_loss_price = entry_price * (1 - self.stop_loss_percent)
                take_profit_price = entry_price * (1 + self.take_profit_percent)

                if current_price <= stop_loss_price or current_price >= take_profit_price:
                    capital = position * current_price
                    position = 0
                    action = "stop_loss" if current_price <= stop_loss_price else "take_profit"
                    self.results["stop_loss_hits" if action == "stop_loss" else "take_profit_hits"] += 1
                    self.results['winning_trades' if action == "take_profit" else 'losing_trades'] += 1
                    self.results['total_trades'] += 1
                    self.trade_log.append({"action": "sell", "price": current_price})


            current_equity = capital + (position * current_price if position > 0 else 0)
            self.equity_curve.append(current_equity)

            if len(self.equity_curve) > 1:
                prev_equity = self.equity_curve[-2]
                return_rate = (current_equity - prev_equity) / prev_equity
                self.returns.append(return_rate)

        final_capital = capital + (position * self.data['Close'].iloc[-1] if position > 0 else 0)
        self.equity_curve.append(final_capital)
        return final_capital, self.results

    def calculate_max_drawdown(self):
        peak = self.equity_curve[0]
        max_drawdown = 0
        for value in self.equity_curve:
            if value > peak:
                peak = value
            drawdown = (peak - value) / peak
            max_drawdown = max(max_drawdown, drawdown)
        return max_drawdown
    def calculate_sharpe_ratio(self):
        if not self.returns:
            return 0
        average_return = sum(self.returns) / len(self.returns)
        return_std_dev = (sum((r - average_return) ** 2 for r in self.returns) / len(self.returns)) ** 0.5
        sharpe_ratio = (average_return - self.risk_free_rate) / return_std_dev if return_std_dev != 0 else 0
        return sharpe_ratio


In [29]:
backtester = Backtest(data, stop_loss_percent=0.02, take_profit_percent=0.03, initial_capital=1000,risk_free_rate=0.01)
final_capital, results = backtester.execute_backtest()
profit = final_capital - backtester.initial_capital
max_drawdown = backtester.calculate_max_drawdown()
sharpe_ratio = backtester.calculate_sharpe_ratio()

In [30]:
print(f"Final Capital: {final_capital:.2f}")
print(f"Profit: {profit:.2f}")
print(f"Total Trades: {results['total_trades']}")
print(f"Winning Trades: {results['winning_trades']}")
print(f"Losing Trades: {results['losing_trades']}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Final Capital: 2176.29
Profit: 1176.29
Total Trades: 18
Winning Trades: 8
Losing Trades: 10
Maximum Drawdown: 56.47%
Sharpe Ratio: 0.09
